In [2]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
from tqdm.autonotebook import tqdm, trange
import time

/home/caym/Programs/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [6]:
data = pd.read_csv('data/post_data.csv')
data = data.dropna()
data = data.reset_index()
data.drop(['index'], axis=1, inplace=True)

In [7]:
df = data[:]

In [8]:
from deeppavlov import configs, build_model
ner_model = train_model(configs.ner.ner_rus_bert, , download=True)
ner_model = build_model(configs.ner.ner_rus_bert)

2019-12-13 05:54:05.123 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2019-12-13 05:54:08.76 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /home/caym/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/caym/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/caym/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/caym/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


2019-12-13 05:54:11.43 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/caym/.deeppavlov/models/ner_rus_bert/tag.dict]


Using TensorFlow backend.




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
I

2019-12-13 05:54:35.166 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/caym/.deeppavlov/models/ner_rus_bert/model]



INFO:tensorflow:Restoring parameters from /home/caym/.deeppavlov/models/ner_rus_bert/model


In [10]:
f = open('data/pavlov.txt', 'w')
for i in tqdm(list(df['text'])):
    s = ''
    tok = i.split(' ')
    try:
        if len(tok) < 100:
            res = ner_model([i])
            for j in range(len(res[1][0])):
                if res[1][0][j] == 'B-LOC':
                    s += res[0][0][j] + ' '
            f.write(s + '\n')
        else:
            res = ner_model([" ".join(tok[:100])])
            for j in range(len(res[1][0])):
                if res[1][0][j] == 'B-LOC':
                    s += res[0][0][j] + ' '
            f.write(s + '\n')
    except:
        f.write('\n')
f.close()

In [8]:
pavlov = []
f = open('data/pavlov.txt', 'r')
pavlov = [line.strip() for line in f]

In [9]:
data['pavlov'] = pavlov

In [10]:
data.to_csv('data/new_with_street.csv', index=None)

'Большеохтинском'

In [96]:
text = 'Санкт-Петербург, ул. Планерная'

In [36]:
f = open('data/street_list_spb(without type).txt', 'r')
l = [line.strip() for line in f]

In [11]:
k = 0
o = []
for i in tqdm(list(df['text'])):
    sample = process_text(i)
    p = []
    for j in l:
        if sample.find(stemmer.stem(j)) > 0:
            p.append(j)
    o.append(p)

In [82]:
d = 0
for i in o:
    if len(i)>0:
        d+=1
print(d)

87


In [54]:
extractor = AddressExtractor()
sample = process_text(text)
matches = extractor(text)
for match in matches:
    print(match.fact)

Address(parts=[Settlement(name='Санкт-Петербург', type=None), Street(name='Планерная', type='улица'), Building(number='5', type=None)])


In [37]:
df['text'][8]

'1. Эвакуировали 384 лицей. (в 14:30 всех запустили назад) \n2. Эвакуировали гимназию 405\n3. В 14:00 эвакуировали 98 школу\n4. Эвакуировали гимназию 399\n5. Эвакуировали гимназию 41\n6. 295 гимназию эвакуируют.\n\nЭвакуировали учащихся из школы № 553 Фрунзенского района на улице Ярослава Гашека, 4, корп. 4'

In [38]:
k = 0
f = open('street.txt', 'w')
extractor = AddressExtractor()
for i in tqdm(list(df['text'])):
    sample = process_text(i)
    matches = extractor(sample)
    s = ''
    for match in matches:
        for j in range(0, len(match.fact.parts)):
            try:
                if match.fact.parts[j].type is None:
                    try:
                        s += match.fact.parts[j].number + ' '
                        continue
                    except:
                        s += match.fact.parts[j].name + ' '
                        continue
                try:
                    if match.fact.parts[j].number is None:
                        break
                    s += match.fact.parts[j].type + ' ' + match.fact.parts[j].number + ' '
                except:
                    if match.fact.parts[j].name is None:
                        continue
                s += match.fact.parts[j].type + ' ' + match.fact.parts[j].name + ' '
            except:
                continue
    f.write(s + '\n')
    if len(matches)>0:
        k += 1
f.close()

In [30]:
sample = process_text('Россия Санкт-Петербург Ботаническая улица 66')
matches = extractor(sample)
s = ''
for match in matches:
    for j in range(0, len(match.fact.parts)):
        try:
            if match.fact.parts[j].type is None:
                try:
                    s += match.fact.parts[j].number + ' '
                    continue
                except:
                    s += match.fact.parts[j].name + ' '
                    continue
            try:
                if match.fact.parts[j].number is None:
                    break
                s += match.fact.parts[j].type + ' ' + match.fact.parts[j].number + ' '
            except:
                if match.fact.parts[j].name is None:
                    continue
            s += match.fact.parts[j].type + ' ' + match.fact.parts[j].name + ' '
        except:
            continue

In [31]:
s

'Санкт-Петербург улица Ботаническая 66 '

In [112]:
process_text(text)

'На Комсомола улице дома 10 уехал светофор второй середине перекрестка'

In [124]:
extractor = AddressExtractor()
matches = extractor(process_text(list(df['text'])[0]))
for match in matches:
    print(match.span, match.fact)

In [125]:
len(matches)

0

In [94]:
match.fact.parts

[Street(name='Комсомола',
        type='улица'), Building(number='10',
          type='дом')]

In [183]:
s = match.fact.parts[0].name + ' ' + match.fact.parts[0].type

In [184]:
for i in range(0, len(match.fact.parts)):
    if match.fact.parts[i].type is None:
        s += match.fact.parts[i].number
    else:
        s += match.fact.parts[i].type + ' ' + match.fact.parts[i].number + ' '

TypeError: can only concatenate str (not "NoneType") to str

In [185]:
s

'Лиговском проспект'

In [82]:
match.fact.parts[2].type is None

True

In [86]:
f = open('street.txt', 'r')
l = [line.strip() for line in f]

In [88]:
len(l)

58123

In [89]:
data.shape

(58123, 3)

In [90]:
data['natasha'] = l

In [92]:
data.loc[data['natasha'] != '']

,id,date,text,natasha
8,12341152,2019-12-09 11:18:58,1. Эвакуировали 384 лицей. (в 14:30 всех запус...,район Фрунзенского улица Ярослава Гашека 4 кор...
39,12338074,2019-12-08 15:58:13,"Сегодня в 8 утра, на Кантемировской улице, мо...",улица Кантемировской 9
52,12336390,2019-12-08 06:53:59,Сегодня в 3 часа ночи на Будапештской ул. 95 к...,улица Будапештской 95 корпус 1
53,12336388,2019-12-08 06:52:39,"Туристская, 28.\n5ого декабря с утра обнаружил...",улица Туристкая 26
56,12335402,2019-12-07 19:12:55,Новости города за сегодняшний день:\n\n1. Липо...,набережная Петроградской 6
...,...,...,...,...
58046,4809342,2016-09-11 23:52:04,6 сентября был угнан мой любимый Форд Фокус М7...,бульвар Сиреневом 6
58052,4807823,2016-09-11 19:45:50,На Ропшинском шоссе 08 09.2016\n(Смотреть с 0;...,шоссе На Ропшинском 08
58057,4807684,2016-09-11 19:35:07,ДТП на Новоприозерском шоссе 11.09.2016\nК нов...,шоссе Новоприозерском 11
58095,4804611,2016-09-11 10:29:17,Угнан автомобиль Тойота Хайлендр 2014 г серебр...,район Приморский улица Сизова Испытателей 10
